# **Zhanibek Meiramkan ADA-2404M Assignment 2**

# Задача 1

Гетерогенная параллелизация это подход при котором в одной программе используются разные вычислительные устройства чаще всего cpu и gpu. Смысл в том что разные части задачи выполняются на том устройстве которое лучше всего для этого подходит.

Cpu и gpu работают по разному. Cpu имеет небольшое количество ядер но они универсальные и хорошо подходят для управления программой логики условий и ветвлений. Gpu наоборот содержит очень большое количество простых ядер и лучше всего справляется с одинаковыми операциями над большими массивами данных.

Основное преимущество гетерогенной параллелизации это ускорение работы программы. Тяжелые вычисления выполняются на gpu а cpu занимается управлением и общей логикой. За счет этого ресурсы компьютера используются эффективнее и обработка больших объемов данных занимает меньше времени.

Гетерогенная параллелизация активно используется в реальных приложениях. Она применяется в компьютерной графике и играх в машинном обучении и нейросетях при обработке изображений и видео а также в научных расчетах и моделировании. Например в задачах машинного обучения gpu выполняет вычисления с матрицами а cpu управляет процессом обучения.

# Задача 2

In [45]:
%%writefile task2.cpp
#include <iostream> // ввод и вывод
#include <cstdlib>  // rand, srand
#include <ctime> // time
#include <chrono> // измерение времени
#include <omp.h>  // openmp

using namespace std;  // чтоб не писать std::

void task2() { // функция
    const int N = 10000; // размер массива
    int* arr = new int[N]; // динамический массив
    srand(time(0)); // инициализация рандома

    for (int i = 0; i < N; i++) { // заполняем массив
        arr[i] = rand() % 10000 + 1; // числа от 1 до 10000
    }

  // последовательный поиск
    auto s1 = chrono::high_resolution_clock::now(); // старт seq
    int min_seq = arr[0]; // минимум
    int max_seq = arr[0]; // максимум

    for (int i = 1; i < N; i++) { // проход по массиву
        if (arr[i] < min_seq) min_seq = arr[i]; // минимум
        if (arr[i] > max_seq) max_seq = arr[i]; // максимум
    }
 // параллельный поиск
    auto e1 = chrono::high_resolution_clock::now(); // конец seq
    double time_seq = chrono::duration<double, milli>(e1 - s1).count(); // время

    auto s2 = chrono::high_resolution_clock::now(); // старт par
    int min_par = 10001; // минимум
    int max_par = 0; // максимум

#pragma omp parallel
    { // параллельная область
        int local_min = 10001; // локальный минимум
        int local_max = 0; // локальный максимум

#pragma omp for nowait
        for (int i = 0; i < N; i++) { // делим массив
            if (arr[i] < local_min) local_min = arr[i]; // локальный минимум
            if (arr[i] > local_max) local_max = arr[i]; // локальный максимум
        }

#pragma omp critical
        { // обновляем результат
            if (local_min < min_par) min_par = local_min; // общий минимум
            if (local_max > max_par) max_par = local_max; // общий максимум
        }
    }

    auto e2 = chrono::high_resolution_clock::now(); // конец par
    double time_par = chrono::duration<double, milli>(e2 - s2).count(); // время
// выводы
    cout << "последовательно:\n"; // вывод
    cout << "минимум = " << min_seq << "\n"; // минимум
    cout << "максимум = " << max_seq << "\n"; // максимум
    cout << "время = " << time_seq << " мс\n\n"; // время

    cout << "параллельно:\n"; // вывод
    cout << "минимум = " << min_par << "\n"; // минимум
    cout << "максимум = " << max_par << "\n"; // максимум
    cout << "время = " << time_par << " мс\n"; // время

    delete[] arr; // освобождаем память
    arr = nullptr; // обнуляем указатель
}

int main() { // main
    task2(); // запуск задания
    return 0; // выход
}

Overwriting task2.cpp


In [46]:
!g++ -fopenmp task2.cpp -o task2
!./task2



последовательно:
минимум = 2
максимум = 9999
время = 0.01954 мс

параллельно:
минимум = 2
максимум = 9999
время = 0.15751 мс


# Задача 3

In [49]:
%%writefile task3.cpp
#include <iostream> // ввод вывод
#include <cstdlib> // rand srand
#include <ctime> // time
#include <chrono> // время
#include <omp.h> // openmp

using namespace std;  // чтоб не писать std::

void fill_array(int* arr, int n) { // заполняем массив
    for (int i = 0; i < n; i++) { // цикл
        arr[i] = rand() % 10000 + 1; // случайные числа
    }
}

void copy_array(int* to, int* from, int n) { // копия массива
    for (int i = 0; i < n; i++) { // цикл
        to[i] = from[i]; // копируем
    }
}

void selection_sort_seq(int* arr, int n) { // выбором seq
    for (int i = 0; i < n - 1; i++) { // идем по массиву
        int min_idx = i; // считаем что это минимум

        for (int j = i + 1; j < n; j++) { // ищем меньше
            if (arr[j] < arr[min_idx]) { // если нашли
                min_idx = j; // запоминаем
            }
        }

        int temp = arr[i]; // временно сохраняем
        arr[i] = arr[min_idx]; // ставим минимум
        arr[min_idx] = temp; // возвращаем обратно
    }
}

void selection_sort_par(int* arr, int n) { // выбором par
    for (int i = 0; i < n - 1; i++) { // внешний цикл
        int min_val = arr[i]; // минимум
        int min_idx = i; // индекс минимума

#pragma omp parallel
        { // параллельная зона
            int local_min = min_val; // локальный минимум
            int local_idx = min_idx; // локальный индекс

#pragma omp for nowait
            for (int j = i + 1; j < n; j++) { // делим массив
                if (arr[j] < local_min) { // если меньше
                    local_min = arr[j]; // обновляем
                    local_idx = j; // обновляем
                }
            }

#pragma omp critical
            { // обновляем общий минимум
                if (local_min < min_val) {
                    min_val = local_min;
                    min_idx = local_idx;
                }
            }
        }

        int temp = arr[i]; // временно сохраняем
        arr[i] = arr[min_idx]; // ставим минимум
        arr[min_idx] = temp; // возвращаем обратно
    }
}

void test_size(int n) { // тест размера
    int* base = new int[n]; // исходный массив
    int* arr = new int[n]; // рабочий массив

    fill_array(base, n); // заполняем

    copy_array(arr, base, n); // копия
    auto s1 = chrono::high_resolution_clock::now(); // старт seq
    selection_sort_seq(arr, n); // сортировка
    auto e1 = chrono::high_resolution_clock::now(); // конец seq
    double t_seq = chrono::duration<double, milli>(e1 - s1).count(); // время

    copy_array(arr, base, n); // копия
    auto s2 = chrono::high_resolution_clock::now(); // старт par
    selection_sort_par(arr, n); // сортировка
    auto e2 = chrono::high_resolution_clock::now(); // конец par
    double t_par = chrono::duration<double, milli>(e2 - s2).count(); // время

    cout << "\nразмер = " << n << "\n"; // вывод
    cout << "последовательно: " << t_seq << " мс\n"; // seq
    cout << "параллельно:     " << t_par << " мс\n"; // par

    delete[] base; // чистим
    delete[] arr; // чистим
}

int main() { // main
    srand(time(0)); // рандом
    test_size(1000); // тест 1000
    test_size(10000); // тест 10000
    return 0; // выход
}


Overwriting task3.cpp


In [50]:
!g++ -fopenmp task3.cpp -o task3
!./task3



размер = 1000
последовательно: 1.34839 мс
параллельно:     1.39324 мс

размер = 10000
последовательно: 78.5723 мс
параллельно:     60.7425 мс


# Задача 4

In [6]:
%%writefile task4.cu
#include <iostream> // вывод
#include <cstdlib> // rand srand
#include <ctime> // время
#include <chrono> // время
#include <cuda_runtime.h> // cuda

using namespace std; // чтоб не писать std::

__global__ void sort_block(int* data, int n, int block_size) { // сортим кусок на gpu
    int bid = blockIdx.x; // номер блока
    int start = bid * block_size; // начало куска
    int end = start + block_size; // конец куска
    if (end > n) end = n; // чтобы не вылезти за массив

    for (int i = start; i < end; i++) { // простой пузырек
        for (int j = start; j + 1 < end; j++) { // бегаем по соседям
            if (data[j] > data[j + 1]) { // если перепутаны
                int temp = data[j]; // временно
                data[j] = data[j + 1]; // меняем
                data[j + 1] = temp; // обратно
            }
        }
    }
}

void merge_cpu(int* arr, int n, int block_size) { // сливаем куски на cpu
    for (int size = block_size; size < n; size *= 2) { // размер кусков растет
        for (int left = 0; left < n; left += 2 * size) { // берем 2 куска
            int mid = left + size; // середина
            int right = left + 2 * size; // конец
            if (mid > n) mid = n; // подрезаем границу
            if (right > n) right = n; // тоже подрезаем
            if (mid >= right) continue; // если второго куска нет

            int len = right - left; // длина участка
            int* temp = new int[len]; // временный массив
            int i = left; // левый кусок
            int j = mid; // правый кусок
            int k = 0; // позиция в temp

            while (i < mid && j < right) { // пока есть с двух сторон
                if (arr[i] < arr[j]) temp[k++] = arr[i++]; // берем слева
                else temp[k++] = arr[j++]; // берем справа
            }
            while (i < mid) temp[k++] = arr[i++]; // дописываем левый хвост
            while (j < right) temp[k++] = arr[j++]; // дописываем правый хвост

            for (int x = 0; x < len; x++) arr[left + x] = temp[x]; // копируем назад
            delete[] temp; // освобождаем память
        }
    }
}

void test_size(int n) { // тест размера
    int* arr = new int[n]; // массив на cpu
    for (int i = 0; i < n; i++) arr[i] = rand() % 10000 + 1; // заполняем числами

    int* d_arr = nullptr; // массив на gpu
    cudaMalloc(&d_arr, n * (int)sizeof(int)); // память на gpu
    cudaMemcpy(d_arr, arr, n * (int)sizeof(int), cudaMemcpyHostToDevice); // копируем на gpu

    int block_size = 256; // сколько элементов в куске
    int blocks = (n + block_size - 1) / block_size; // сколько блоков нужно

    auto s = chrono::high_resolution_clock::now(); // старт времени

    sort_block<<<blocks, 1>>>(d_arr, n, block_size); // каждый блок сортит свой кусок
    cudaDeviceSynchronize(); // ждем пока gpu закончит

    cudaMemcpy(arr, d_arr, n * (int)sizeof(int), cudaMemcpyDeviceToHost); // обратно на cpu
    merge_cpu(arr, n, block_size); // сливаем все в один массив

    auto e = chrono::high_resolution_clock::now(); // конец времени
    double t = chrono::duration<double, milli>(e - s).count(); // время в мс

    cout << "размер = " << n << ", время = " << t << " мс\n"; // вывод

    cudaFree(d_arr); // чистим gpu память
    delete[] arr; // чистим cpu память
    arr = nullptr; // на всякий
}

int main() { // main
    srand(time(0)); // рандом
    test_size(10000); // тест 10 000
    test_size(100000); // тест 100 000
    return 0; // выход
}



Overwriting task4.cu


In [7]:
!nvcc task4.cu -o task4
!./task4




размер = 10000, время = 8.1685 мс
размер = 100000, время = 5.21447 мс
